In [1]:
import numpy as np
import pandas as pd
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
pd.set_option('display.precision', 4)
np.set_printoptions(suppress=True, precision=4)
#leggiamo i dati all'interno di un DataFrame
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'
data = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

<ipython-input-1-64a8bcc45ecb>:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
def add_lags(data, ric, lags, window=50):
    cols = []
    df = pd.DataFrame(data[ric])
    df.dropna(inplace=True)
    df['r'] = np.log(df / df.shift())
    #SMA
    df['sma'] = df[ric].rolling(window).mean()
    #rolling minimum
    df['min'] = df[ric].rolling(window).min()  
    #rolling maximum
    df['max'] = df[ric].rolling(window).max()
    #momentum come media dei rendimenti log
    df['mom'] = df['r'].rolling(window).mean()
    #rolling volatility
    df['vol'] = df['r'].rolling(window).std()  
    df.dropna(inplace=True)
    #direzione come feature binaria
    df['d'] = np.where(df['r'] > 0, 1, 0)  
    features = [ric, 'r', 'd', 'sma', 'min', 'max', 'mom', 'vol']
    for f in features:
        for lag in range(1, lags + 1):
            col = f'{f}_lag_{lag}'
            df[col] = df[f].shift(lag)
            cols.append(col)
    df.dropna(inplace=True)
    return df, cols

lags = 5
dfs = {}
for ric in data:
    df, cols = add_lags(data, ric, lags)
    dfs[ric] = df.dropna(), cols

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

for ric in data:
    model = MLPClassifier(hidden_layer_sizes=[512],
                          random_state=100,
                          max_iter=1000,
                          early_stopping=True,
                          validation_fraction=0.15,
                          shuffle=False)
    df, cols = dfs[ric]
    #normalizza i dati delle features
    df[cols] = (df[cols] - df[cols].mean()) / df[cols].std()
    model.fit(df[cols], df['d'])
    pred = model.predict(df[cols])
    acc = accuracy_score(df['d'], pred)
    print(f'IN-SAMPLE | {ric:7s} | acc={acc:.4f}')

IN-SAMPLE | AAPL.O  | acc=0.5510
IN-SAMPLE | MSFT.O  | acc=0.5376
IN-SAMPLE | INTC.O  | acc=0.5607
IN-SAMPLE | AMZN.O  | acc=0.5559
IN-SAMPLE | GS.N    | acc=0.5794
IN-SAMPLE | SPY     | acc=0.5729
IN-SAMPLE | .SPX    | acc=0.5941
IN-SAMPLE | .VIX    | acc=0.6940
IN-SAMPLE | EUR=    | acc=0.5766
IN-SAMPLE | XAU=    | acc=0.5672
IN-SAMPLE | GDX     | acc=0.5847
IN-SAMPLE | GLD     | acc=0.5567


In [7]:
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential

np.random.seed(100)
tf.random.set_seed(100)

def create_model(problem='regression'): 
  model = Sequential()
  model.add(Dense(512, input_dim=len(cols),
                    activation='relu'))
  if problem == 'regression':
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='adam')
  else:
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
  return model

for ric in data:
    model = create_model('classification')
    df, cols = dfs[ric]
    #normalizza i dati delle features
    df[cols] = (df[cols] - df[cols].mean()) / df[cols].std()
    model.fit(df[cols], df['d'], epochs=50, verbose=False)
    pred = np.where(model.predict(df[cols]) > 0.5, 1, 0)
    acc = accuracy_score(df['d'], pred)
    print(f'IN-SAMPLE | {ric:7s} | acc={acc:.4f}')

77/77 [==============================] - 0s 2ms/step
IN-SAMPLE | AAPL.O  | acc=0.7017
77/77 [==============================] - 0s 2ms/step
IN-SAMPLE | MSFT.O  | acc=0.6912
77/77 [==============================] - 0s 1ms/step
IN-SAMPLE | INTC.O  | acc=0.7026
77/77 [==============================] - 0s 2ms/step
IN-SAMPLE | AMZN.O  | acc=0.6786
77/77 [==============================] - 0s 1ms/step
IN-SAMPLE | GS.N    | acc=0.6883
77/77 [==============================] - 0s 1ms/step
IN-SAMPLE | SPY     | acc=0.6826
77/77 [==============================] - 0s 1ms/step
IN-SAMPLE | .SPX    | acc=0.6940
77/77 [==============================] - 0s 1ms/step
IN-SAMPLE | .VIX    | acc=0.7538
77/77 [==============================] - 0s 2ms/step
IN-SAMPLE | EUR=    | acc=0.6684
77/77 [==============================] - 0s 2ms/step
IN-SAMPLE | XAU=    | acc=0.7005
77/77 [==============================] - 0s 2ms/step
IN-SAMPLE | GDX     | acc=0.6904
77/77 [==============================] - 0s 1ms/step
I

In [9]:
def train_test_model(model):
    for ric in data:
        df, cols = dfs[ric]
        split = int(len(df) * 0.85)
        train = df.iloc[:split].copy()
        #Le statistiche del set di dati di addestramento vengono utilizzate per la normalizzazione.
        mu, std = train[cols].mean(), train[cols].std()
        train[cols] = (train[cols] - mu) / std
        model.fit(train[cols], train['d'])
        test = df.iloc[split:].copy()
        test[cols] = (test[cols] - mu) / std
        pred = model.predict(test[cols])
        acc = accuracy_score(test['d'], pred)
        print(f'OUT-OF-SAMPLE | {ric:7s} | acc={acc:.4f}')

model_mlp = MLPClassifier(hidden_layer_sizes=[512],
                          random_state=100,
                          max_iter=1000,
                          early_stopping=True,
                          validation_fraction=0.15,
                          shuffle=False)

train_test_model(model_mlp)

OUT-OF-SAMPLE | AAPL.O  | acc=0.4432
OUT-OF-SAMPLE | MSFT.O  | acc=0.4595
OUT-OF-SAMPLE | INTC.O  | acc=0.5000
OUT-OF-SAMPLE | AMZN.O  | acc=0.5270
OUT-OF-SAMPLE | GS.N    | acc=0.4838
OUT-OF-SAMPLE | SPY     | acc=0.4811
OUT-OF-SAMPLE | .SPX    | acc=0.5027
OUT-OF-SAMPLE | .VIX    | acc=0.5676
OUT-OF-SAMPLE | EUR=    | acc=0.4649
OUT-OF-SAMPLE | XAU=    | acc=0.5514
OUT-OF-SAMPLE | GDX     | acc=0.5162
OUT-OF-SAMPLE | GLD     | acc=0.4946


In [12]:
from sklearn.ensemble import BaggingClassifier

#stimatore base
base_estimator = MLPClassifier(hidden_layer_sizes=[256],
                                random_state=100,
                                max_iter=1000,
                                early_stopping=True,
                                validation_fraction=0.15,
                                shuffle=False)  

model_bag = BaggingClassifier(base_estimator=base_estimator,
                              #numero di stimatori usati
                              n_estimators=35,
                              #percentuale massima di dati di addestramento utilizzati per 
                              #stimatore
                              max_samples=0.25,
                              #numero massimo di features usate per stimatore
                              max_features=0.5,
                              #se eseguire il bootstrap (riutilizzo) dei dati  
                              bootstrap=False,
                              #se eseguire il bootstrap (riutilizzo) delle funzionalità
                              bootstrap_features=True,
                              #numero di lavori paralleli
                              n_jobs=8, 
                              random_state=100
                              )

train_test_model(model_bag)

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


OUT-OF-SAMPLE | AAPL.O  | acc=0.5000


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


OUT-OF-SAMPLE | MSFT.O  | acc=0.5703


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


...

OUT-OF-SAMPLE | XAU=    | acc=0.5351


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


OUT-OF-SAMPLE | GDX     | acc=0.5054


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


OUT-OF-SAMPLE | GLD     | acc=0.5189


In [13]:
url = 'http://hilpisch.com/aiif_eikon_id_data.csv'

data = pd.read_csv(url, index_col=0, parse_dates=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5529 entries, 2019-03-01 00:00:00 to 2020-01-01 00:00:00
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL.O  3384 non-null   float64
 1   MSFT.O  3378 non-null   float64
 2   INTC.O  3275 non-null   float64
 3   AMZN.O  3381 non-null   float64
 4   GS.N    1686 non-null   float64
 5   SPY     3388 non-null   float64
 6   .SPX    1802 non-null   float64
 7   .VIX    2959 non-null   float64
 8   EUR=    5429 non-null   float64
 9   XAU=    5149 non-null   float64
 10  GDX     3173 non-null   float64
 11  GLD     3351 non-null   float64
dtypes: float64(12)
memory usage: 561.5 KB


In [14]:
data.head()

,AAPL.O,MSFT.O,INTC.O,AMZN.O,GS.N,SPY,.SPX,.VIX,EUR=,XAU=,GDX,GLD
Date,,,,,,,,,,,,
2019-03-01 00:00:00,173.15,112.09,53.03,1640.4,NaN,278.59,NaN,NaN,1.1373,1314.0663,22.24,124.07
2019-03-01 01:00:00,173.44,112.24,NaN,1642.8,NaN,278.80,NaN,NaN,1.1369,1312.9100,22.24,124.08
2019-03-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1372,1314.3636,NaN,NaN
2019-03-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1367,1313.2452,NaN,NaN
2019-03-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1369,1313.1746,NaN,NaN


In [15]:
lags = 5

dfs = {}

for ric in data:
  df, cols = add_lags(data, ric, lags)
  dfs[ric] = df, cols

train_test_model(model_mlp)

OUT-OF-SAMPLE | AAPL.O  | acc=0.5420
OUT-OF-SAMPLE | MSFT.O  | acc=0.4930
OUT-OF-SAMPLE | INTC.O  | acc=0.5549
OUT-OF-SAMPLE | AMZN.O  | acc=0.4709
OUT-OF-SAMPLE | GS.N    | acc=0.5184
OUT-OF-SAMPLE | SPY     | acc=0.4860
OUT-OF-SAMPLE | .SPX    | acc=0.5019
OUT-OF-SAMPLE | .VIX    | acc=0.4885
OUT-OF-SAMPLE | EUR=    | acc=0.5130
OUT-OF-SAMPLE | XAU=    | acc=0.4824
OUT-OF-SAMPLE | GDX     | acc=0.4765
OUT-OF-SAMPLE | GLD     | acc=0.5455


In [16]:
train_test_model(model_bag)

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


OUT-OF-SAMPLE | AAPL.O  | acc=0.5660


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


OUT-OF-SAMPLE | MSFT.O  | acc=0.5551


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


...

OUT-OF-SAMPLE | XAU=    | acc=0.5229


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


OUT-OF-SAMPLE | GDX     | acc=0.5107


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


OUT-OF-SAMPLE | GLD     | acc=0.5475
